In [ ]:
from pathlib import Path

In [ ]:
import fiona
data = fiona.open("../../data/shp_bdry_votingdistricts/bdry_votingdistricts_4326.shp")
data.schema

In [ ]:
data[1]

In [ ]:
import geopandas as gpd
df = gpd.read_file("../../data/shp_bdry_votingdistricts/bdry_votingdistricts_4326.shp")
df.plot(figsize=[20,30])

In [ ]:
file = Path("../../data/shp_bdry_counties_in_minnesota/mn_county_boundaries_500.shp")
data = gpd.read_file(file)
reprojected = data.to_crs("EPSG:4326")
counties = ["Anoka", "Carver", "Dakota", "Hennepin", "Ramsey", "Scott", "Washington"]
county_mask = reprojected["CTY_NAME"].isin(counties)
county_data = reprojected.loc[county_mask]
county_data.plot()

In [ ]:
from shapely import wkt
from shapely.ops import cascaded_union
tc_metro = cascaded_union(county_data.geometry)
tc_metro

In [ ]:
metro_districts = df.loc[df.geometry.apply(lambda x: tc_metro.contains(x))]
metro_districts.geometry.boundary.plot(color=None,edgecolor='k',linewidth = 2)

In [ ]:
#something looks off with this so I will just use bounds on the metro instead to reduce number of precincts to iterate through
points = tc_metro.bounds
bounds_polygon = f"POLYGON(({points[0]} {points[1]}, {points[0]} {points[3]}, {points[2]} {points[3]}, {points[2]} {points[1]}, {points[0]} {points[1]}))"
bounds_polygon = wkt.loads(bounds_polygon)
metro_districts = df.loc[df.geometry.apply(lambda x: bounds_polygon.contains(x))]
metro_districts.geometry.boundary.plot(color=None,edgecolor='k',linewidth = 2)

In [ ]:
files = [file.name for file in Path(".").glob("../../data/shp_bdry_electionresults_2012_2020/*4326.shp")]
import re
print(files)
print(re.findall("(20[0-9]{2,})", files[0])[0])


In [ ]:
# Election results
import pandas as pd
from pathlib import Path
import glob
import fiona
for file in Path(".").glob("../../data/shp_bdry_electionresults_2012_2020/*4326.shp"):
    print(file.name + "\n")
    data = fiona.open(file)
    # election_results = pd.read_excel("../../data/2020-general-federal-state-results-by-precinct-official.xlsx", sheet_name='Precinct-Results', engine="openpyxl")
    # election_results = election_results.dropna(how="all", axis=0).iloc[:-1]
    print([key for key in data.schema["properties"].keys() if "." in key])
    if "2016" in file.name:
        break

In [ ]:
data[0]["properties"]

In [ ]:
election_results[]

In [ ]:
len(df)

In [ ]:
import geopandas as gpd
import pandas as pd
df = gpd.read_file("../../data/shp_bdry_votingdistricts/bdry_votingdistricts_4326.shp")
df["PCTCODE"] = pd.to_numeric(df["PCTCODE"])
cols_to_use = list(election_results.columns.difference(df.columns))

merged = pd.merge(df, election_results[cols_to_use + ["VTDID"]], on="VTDID", how="left")
merged = merged.set_crs(epsg=4326)
def convert_code(code):
    code = str(code)
    return f"0000{code}"[-4:]
merged["PCTCODE"] = merged["PCTCODE"].apply(convert_code)
merged

In [ ]:
merged = merged.drop(columns="TABMODEL.1")

In [ ]:
merged.to_file("../../data/shp_bdry_electionresults_2012_2020/general_election_results_by_precinct_2020_4326.shp")